In [2]:
import numpy as np
from scipy.linalg import svd, qr, eig, eigh, pinv
from scipy.linalg import inv as inv
from scipy.sparse.linalg import svds
from POD import import_data
import os


In [14]:
current_directory = os.getcwd()
time, x, z, u, w, T, umean = import_data(f'/home/julielimonet/Documents/Stage_M2/')

In [20]:
np.shape(u)
h, l = np.shape(x)
m = len(time)
# reshape
U = np.reshape(u, (m, h * l))

4131


In [32]:
print(np.shape(U), np.shape(np.swapaxes(U,0,1)))
x = np.swapaxes(U,0,1)
x, y = x[:, :-1], x[:, 1:]

# First svd
U, Sigma, W = svd(x, full_matrices=True)
W = W.T.conj()

(4131, 4131) (4131, 4131)


In [33]:
print(np.shape(Sigma))

(4130,)


In [27]:
rtol = 1e-7
print(np.diag(Sigma >= Sigma.max()*rtol))

[[ True False False ... False False False]
 [False  True False ... False False False]
 [False False  True ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]


In [34]:
S = U.T.conj() @ y @ W @ pinv(np.diag(Sigma))

In [46]:
print(np.shape(Sigma))

(4130,)


In [40]:
print(np.shape(S))


(4131, 4130)


In [52]:
eigenvalues, eigenvectors = eig(np.dot(S, S.T))
real_eigenvectors = np.real(eigenvectors)

In [53]:
print(np.shape(eigvectors))
print(np.shape(U))

(4130, 4130)
(4131, 4131)


In [54]:
rank = len(U[0])
modes = []
[modes.append([]) for i in range(rank)]
for i in range(rank):
  modes[i] = np.real(eigenvectors[:,i])
  modes[i] = modes[i].reshape((h,l)) 
  print(modes[i].shape)

(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(81, 51)
(

In [51]:
print(h*l)

4131
